In [6]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
import requests
from langchain_community.tools import DuckDuckGoSearchRun


In [7]:
# agent specific 
# from langchain.agents import create_react_agent
from langchain_classic.agents import create_react_agent, AgentExecutor
from langchain_classic import hub

hwchase_prompt = hub.pull('hwchase17/react') # pulls the standard ReAct(REasoning + ACTing) agent prompt
hwchase_prompt

PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'}, template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}')

In [8]:
# tool
search_tool = DuckDuckGoSearchRun()
result = search_tool.invoke('Top news in India')

In [17]:
@tool
def get_weather_data(city: str) -> str:
    """API to get current weather for a city name"""
    url = f"http://api.weatherstack.com/current?access_key=ef563678a460ddfe3430000ac6f31d1b&query={city}"
    return(requests.get(url).json())

# testing
# get_weather_data.invoke({'city':'Dubai'})

In [18]:
# llm
llm = ChatOpenAI()

In [19]:
# create an agent - agent "thinks"
agent = create_react_agent(
    llm=llm,
    tools=[search_tool, get_weather_data],
    prompt=hwchase_prompt
)

In [20]:
# wrap it in agent executor - executor "executes"
agent_executor = AgentExecutor(
    agent=agent,
    tools=[search_tool, get_weather_data],
    verbose=True   # to print the thought process
)

In [21]:
agent_executor.invoke({'input' : "Tell me about Pune"})



> Entering new AgentExecutor chain...
I should use the duckduckgo_search tool to get information about Pune.
Action: duckduckgo_search
Action Input: "Pune"Pune skylinePune is a prominent city in the Indian state of Maharashtra, situated in the Deccan Plateau region of western India, and functions as the administrative headquarters of Pune... O que fazer: Pune . Confira atrações e atividades imperdíveis: Darshan Museum, Dagadusheth Halwai Ganapati Temple, Locais religiosos e sagrados, Passeios na natureza e vida selvagem. Pune (Marathi: Puṇē , pronounced [ˈpuɳe] POO-nay ), previously spelled in English as Poona (the official name until 1978), is a city in the state of Maharashtra in the Deccan Plateau in... Pune City Illustration. Old Building Architecture In Pune . Indian Postal History Collection.The Infosys Headquarters in Pune (India) | Arquitectura moderna, Arquitectos, Edificios. Pune , is the second largest city in the state of Maharashtra and the seventh most populous city in 

{'input': 'Tell me about Pune',
 'output': 'Pune is a prominent city in the Indian state of Maharashtra, with an estimated population of 7.4 million. The current weather in Pune is sunny with a temperature of 24°C.'}